In [1]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [6]:
data = pd.read_csv('train.csv', encoding='utf8') 
data = pd.DataFrame(data)

In [3]:
num_classes = len(data['title'])

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(data['label'])
labels = lbl_encoder.transform(data['label'])

In [14]:
data['title']

0                  vieri ronaldo giỏi hơn cristiano_ronaldo
1         phúc_thẩm vụ vn pharma triệu_tập thứ_trưởng bộ...
2                    tử_tù hiến tạng hiến xác khó thực_hiện
3             điểm_chuẩn trường đh văn_hoá tphcm đh hoa_sen
4         5 hành_tinh sáng nhất xếp thẳng hàng trong thá...
                                ...                        
207995    bundesliga cầu_thủ đội bóng khán_giả tôn_trọng...
207996                     dược_thảo tốt gan người cao_tuổi
207997    ngành đường_sắt lắp hơn 1 000 camera đầu_máy g...
207998                        mỹ chiến_hạm eo_biển đài_loan
207999               ông đinh_la_thăng lần khóc tự bào_chữa
Name: title, Length: 208000, dtype: object

In [8]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# the size of the vocabulary in the text data
vocab_size = 2000
# size of the vector space in which words will be embedded
embedding_dim = 16
# length of input sequences
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(data['title'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(data['title'])

# The “pad_sequences” method is used to make all the training text sequences into the same size
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [13]:
sequences

[[1, 882, 1406, 42, 1, 882],
 [660, 1168, 38, 712, 1, 21, 221, 361, 202, 47, 273, 124, 1228, 118],
 [61, 544, 1193, 1694, 1193, 397, 339, 205, 76],
 [44, 252, 35, 498, 236, 198, 857, 498, 135, 1807],
 [49, 92, 358, 255, 22, 980, 776, 15, 4, 159, 59],
 [484, 51, 605, 1329, 306, 228, 413, 704],
 [744, 51, 334, 2, 1, 597, 131],
 [95, 405, 925, 706, 1],
 [948, 1384, 1403, 18, 9, 1369, 424, 1],
 [83, 159, 497, 938, 490, 603, 438, 387, 15, 3, 842, 448],
 [29, 185, 1774, 323, 191, 365, 323, 186, 126, 656, 875],
 [53, 202, 111, 914, 412, 154, 8, 81, 206, 332],
 [158, 930, 38, 61, 230, 102, 364, 687, 1010, 363, 3, 54, 197],
 [398, 494, 370, 3, 842, 448, 399, 1728, 659],
 [378, 1035, 730, 422, 1999, 65, 42, 1099, 237, 1],
 [72, 258, 1006, 508, 1, 782, 23, 65, 428],
 [1376, 637, 413, 65, 22, 5, 7, 705, 239, 58, 56, 171, 66, 1],
 [129, 1126, 1, 255, 1165, 24, 33],
 [5, 7, 204, 35, 1141, 42, 49, 451, 593, 139, 31, 479, 1144],
 [325, 1, 1557, 190, 1456, 310, 479, 80, 695, 1, 361, 107],
 [284, 574, 2

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(LSTM(150))
model.add(Dense(16, activation='relu')) # 16 class to categorize data
model.add(Dense(16, activation='relu')) # 2 times, more exactly
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


In [8]:
epochs = 400
history = model.fit(padded_sequences, np.array(labels), epochs=epochs)

Epoch 1/400
6500/6500 [==============================] - 580s 89ms/step - loss: 2.6303 - accuracy: 0.0766
Epoch 2/400
6500/6500 [==============================] - 597s 92ms/step - loss: 2.5703 - accuracy: 0.0781
Epoch 3/400
6500/6500 [==============================] - 578s 89ms/step - loss: 1.8658 - accuracy: 0.3350
Epoch 4/400
6500/6500 [==============================] - 575s 88ms/step - loss: 1.4062 - accuracy: 0.5470
Epoch 5/400
6500/6500 [==============================] - 571s 88ms/step - loss: 1.2017 - accuracy: 0.6351
Epoch 6/400
6500/6500 [==============================] - 581s 89ms/step - loss: 1.1112 - accuracy: 0.6670
Epoch 7/400
6500/6500 [==============================] - 598s 92ms/step - loss: 1.0364 - accuracy: 0.6891
Epoch 8/400
6500/6500 [==============================] - 583s 90ms/step - loss: 0.9673 - accuracy: 0.7098
Epoch 9/400
6500/6500 [==============================] - 578s 89ms/step - loss: 0.9189 - accuracy: 0.7221
Epoch 10/400
6500/6500 [======================

KeyboardInterrupt: 

In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
def plot_graphs(history):
  plt.title('Loss VS Accuracy')
  plt.plot(history.history['loss'], label='train loss')
  plt.plot(history.history['val_loss'], label='test loss')
  plt.legend()
  
  plt.plot(history.history['accuracy'], label='train accuracy')
  plt.plot(history.history['val_accuracy'], label='test accuracy')
  plt.legend()
  plt.show();
  return None
def loss_graph(history):
  plt.title('Loss')
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show();

  return None

def acc_graph(history):
  plt.title('Accuracy')
  plt.plot(history.history['accuracy'], label='train')
  plt.plot(history.history['val_accuracy'], label='test')
  plt.legend()
  plt.show();

  return None


def evaluate_preds(y_true, y_preds):
    """
    Performs evaluation comparison on y_true labels vs. y_pred labels
    on a classification.
    """
    accuracy = accuracy_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds, average='micro')
    recall = recall_score(y_true, y_preds, average='micro')
    f1 = f1_score(y_true, y_preds, average='micro')
    metric_dict = {"accuracy": round(accuracy, 2),
                   "precision": round(precision, 2),
                   "recall": round(recall, 2),
                   "f1": round(f1, 2)}
    print(f"Acc: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1:.2f}")
    
    return metric_dict

def model_eval(model,a,b,c,d):
  score=model.evaluate(a,b,verbose=0)
  score1=model.evaluate(c,d,verbose=0)
  print("\n")
  print("Model Loss on training data ",score[0])
  print("Model Accuracy on training data: ",score[1])
  print("Model Loss on validation data",score1[0])
  print("Model Accuracy on validation data: ",score1[1])
  print("\n")
  return score,score1

def model_eval(model,a,b,c,d):
  score=model.evaluate(a,b,verbose=0)
  score1=model.evaluate(c,d,verbose=0)
  print("\n")
  print("Model Loss on training data ",score[0])
  print("Model Accuracy on training data: ",score[1])
  print("Model Loss on validation data",score1[0])
  print("Model Accuracy on validation data: ",score1[1])
  print("\n")
  return score,score1



  

In [11]:
# saving model
model.save("model_vietnam_UIT.h5")

# implements binary protocols for serializing and de-serializing a Python object structure.
import pickle

# saving tokenizer
with open('tokenizer_UIT.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#saving label encoder
with open('label_encoder_UIT.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
import random
from difflib import SequenceMatcher
import pickle

def result(inp):
    # load trained model
    model = keras.models.load_model('model_vietnam_UIT.h5')

    # load tokenizer object
    with open('tokenizer_UIT.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    # load label encoder object
    with open('label_encoder_UIT.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                         truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])
    new_tag = tag
    print(new_tag)

In [8]:
result('13 đại học mỹ gần như ứng tuyển là đỗ')

[2]
